In [1]:
#installations
!pip install torchaudio
!sudo apt install -y fluidsynth
!pip install --upgrade pyfluidsynth
!pip install pretty_midi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
#imports
import os
import music21
import pretty_midi
from music21 import midi
import pandas as pd
from IPython.display import Audio
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
from tqdm import tqdm
import statistics
import math, random, time
from IPython.display import Audio
from google.colab import drive
from google.colab import files
import torchaudio
from collections import Counter
import numpy as np
import torch
import torch.nn as nn
import scipy
import shutil

In [4]:
#mount drive, establish data directory
from google.colab import drive
drive.mount('/content/drive')
data_directory = '/content/drive/MyDrive/maestro-v2.0.0/'

Mounted at /content/drive


In [5]:
# define midi reading function
def readMidi(filepath):
  mf = midi.MidiFile()
  mf.open(filepath)
  mf.read()
  mf.close()
  return mf

In [20]:
import os
import shutil
from tqdm import tqdm
import pretty_midi
import numpy as np
import torch
from torch.utils.data import Dataset

class MaestroDataset(Dataset):
    def __init__(self, split, token_dict, directory='/content/drive/MyDrive/maestro-v2.0.0/'):
        import pandas as pd

        self.split = split
        self.token_dict = token_dict
        self.directory = directory.rstrip('/') + '/'
        # read the CSV and filter to our split
        df = pd.read_csv(os.path.join(self.directory, 'maestro-v2.0.0.csv'))
        self.df = df[df['split'] == split].reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def moveFilesToSplitDir(self):
        src_root = self.directory
        dst_root = os.path.join(self.directory, self.split)
        os.makedirs(dst_root, exist_ok=True)

        for fname in self.df['midi_filename']:
            src = os.path.join(src_root, fname)
            if not os.path.isfile(src):
                print(f"Warning: MIDI file not found: {src}")
                continue
            dst = os.path.join(dst_root, os.path.basename(fname))
            shutil.move(src, dst)

    def create_token_sequences(self,
                               pm: pretty_midi.PrettyMIDI,
                               fs: int = 16,
                               seq_len: int = 500,
                               add_bos_eos: bool = True):
        pianoroll = pm.get_piano_roll(fs=fs)
        pianoroll = (pianoroll > 0).astype(int)

        num_frames = pianoroll.shape[1]
        seqs = []
        for start in range(0, num_frames, seq_len):
            end = start + seq_len
            if end > num_frames:
                break
            block = pianoroll[:, start:end]
            # skip mostly-silent windows
            if block.sum() == 0:
                continue

            seq = []
            if add_bos_eos:
                seq.append(self.token_dict['<bos>'])
            for t in range(seq_len):
                notes = np.nonzero(block[:, t])[0]
                if len(notes):
                    seq.append(int(notes[0]))
                else:
                    seq.append(self.token_dict['<rest>'])
            if add_bos_eos:
                seq.append(self.token_dict['<eos>'])
            seqs.append(seq)
        return seqs

    def augment_idxs(self, seq):
        augmented_shift = []
        for shift in range(-12, +13):
            out = []
            for idx in seq:
                # only shift real MIDI notes; leave special tokens alone
                if 0 <= idx < 128:
                    n = idx + shift
                    n = max(0, min(127, n))
                    out.append(n)
                else:
                    out.append(idx)
            augmented_shift.append((shift, out))
        return augmented_shift

    def create_all_token_sequences(self,
                                   fs: int = 16,
                                   seq_len: int = 500):
        split_dir = os.path.join(self.directory, self.split)
        print("-> files in", split_dir, "=", os.listdir(split_dir)[:10])
        all_seqs = []

        for file_name in tqdm(os.listdir(split_dir)):
            if not file_name.lower().endswith(('.mid', '.midi')):
                continue
            path = os.path.join(split_dir, file_name)
            try:
                pm = pretty_midi.PrettyMIDI(path)
                pm.remove_invalid_notes()
                base_name = os.path.basename(file_name)
                compare = self.df['midi_filename'].apply(os.path.basename) == base_name
                print("    looking for", file_name, " -> matched rows:", compare.sum())
                if not compare.any():
                    print(f"Warning: MIDI file not found in CSV: {path}")
                    continue
                composer_label = self.df.loc[compare, 'canonical_composer'].values[0]
                title_label = self.df.loc[compare, 'canonical_title'].values[0]
            except Exception as e:
                print(f"Skipping {file_name}: {e}")
                continue

            seqs = self.create_token_sequences(pm, fs=fs, seq_len=seq_len)
            for s in seqs:
                all_seqs.append({'title': title_label, 'sequence': s, 'composer': composer_label, 'transposition amount': 0})
                for pshift, aug in self.augment_idxs(s):
                    all_seqs.append({'title': title_label, 'sequence': aug, 'composer': composer_label, 'transposition amount': pshift})

        return all_seqs

    def tokens_seqs_to_txt(self, seqs, phase):

        split_dir = os.path.join(self.directory, self.split)
        out_path = os.path.join(split_dir, f"{phase}.txt")
        sorted_seqs = sorted(seqs, key=lambda s: (len(s['title']), s['transposition amount']))
        current_title = None
        current_shift = None
        with open(out_path, 'w') as f:
             for seq in sorted_seqs:
                title = seq['title']
                p_shift = seq['transposition amount']
                if title != current_title or p_shift != current_shift:
                    label = f'{title}, transposition = {p_shift} semitones' if p_shift !=0 else f'{title}'
                    f.write(f'***{label}*** \n')
                    current_title = title
                    current_shift = p_shift
                composer = seq['composer']
                seq = seq['sequence']
                f.write(f'{composer} ' + ' '.join(map(str, seq)) + '\n')


In [21]:
# build token dict once
token_dict = {i:i for i in range(128)}
token_dict.update({'<rest>':128, '<bos>':129, '<eos>':130})
inverse_token_dict = {val:num for val,num in token_dict.items()}

for phase in ['train', 'test', 'validation']:
    ds = MaestroDataset(phase, token_dict,
                        directory='/content/drive/MyDrive/maestro-v2.0.0/')

    ds.moveFilesToSplitDir()
    seqs = ds.create_all_token_sequences(fs=16, seq_len=500)
    ds.tokens_seqs_to_txt(seqs, phase)

-> files in /content/drive/MyDrive/maestro-v2.0.0/train = ['ORIG-MIDI_02_7_6_13_Group__MID--AUDIO_07_R1_2013_wav--3.midi', 'ORIG-MIDI_02_7_8_13_Group__MID--AUDIO_12_R2_2013_wav--2.midi', 'ORIG-MIDI_02_7_6_13_Group__MID--AUDIO_06_R1_2013_wav--3.midi', 'ORIG-MIDI_01_7_8_13_Group__MID--AUDIO_02_R2_2013_wav--3.midi', 'ORIG-MIDI_02_7_8_13_Group__MID--AUDIO_08_R2_2013_wav--2.midi', 'ORIG-MIDI_03_7_8_13_Group__MID--AUDIO_15_R2_2013_wav--2.midi', 'ORIG-MIDI_01_7_8_13_Group__MID--AUDIO_03_R2_2013_wav--2.midi', 'ORIG-MIDI_03_7_6_13_Group__MID--AUDIO_09_R1_2013_wav--4.midi', 'ORIG-MIDI_01_7_8_13_Group__MID--AUDIO_02_R2_2013_wav--2.midi', 'ORIG-MIDI_01_7_8_13_Group__MID--AUDIO_04_R2_2013_wav--3.midi']


  0%|          | 1/914 [00:00<06:55,  2.20it/s]

    looking for ORIG-MIDI_02_7_6_13_Group__MID--AUDIO_07_R1_2013_wav--3.midi  -> matched rows: 1


  0%|          | 2/914 [00:01<12:12,  1.24it/s]

    looking for ORIG-MIDI_02_7_8_13_Group__MID--AUDIO_12_R2_2013_wav--2.midi  -> matched rows: 1


...

    looking for MIDI-Unprocessed_01_R1_2011_MID--AUDIO_R1-D1_06_Track06_wav.midi  -> matched rows: 1


 99%|█████████▉| 130/131 [01:15<00:00,  1.53it/s]

    looking for MIDI-Unprocessed_SMF_16_R1_2004_01-08_ORIG_MID--AUDIO_16_R1_2004_04_Track04_wav.midi  -> matched rows: 1


100%|██████████| 131/131 [01:15<00:00,  1.72it/s]

    looking for MIDI-Unprocessed_XP_01_R1_2004_04-05_ORIG_MID--AUDIO_01_R1_2004_05_Track05_wav.midi  -> matched rows: 1
